In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

def display_noisy_clean_images(noisy_img, clean_img, titles=["Noisy", "Clean"]):
    plt.figure(figsize=(8, 4))
    
    plt.subplot(1, 2, 1)
    plt.imshow(noisy_img, cmap='gray')
    plt.title(titles[0])
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(clean_img, cmap='gray')
    plt.title(titles[1])
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()


In [8]:
class MLP(nn.Module):
    def __init__(self, input_size=64, hidden_size=64, output_size=64, dropout_rate=0.2):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.model(x)

def train_model(train_loader, val_loader, input_size=64, hidden_size=64, lr=1e-3, epochs=50):
    model = MLP(input_size=input_size, hidden_size=hidden_size)
    criterion = nn.MSELoss()
    
    # Weight decay adds L2 regularization
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            preds = model(x_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_losses.append(train_loss / len(train_loader))

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                val_preds = model(x_val)
                val_loss += criterion(val_preds, y_val).item()
        val_losses.append(val_loss / len(val_loader))

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}")

    def plot_loss_curves(train_losses, val_losses):
        plt.figure(figsize=(8, 5))
        plt.plot(train_losses, label="Train Loss")
        plt.plot(val_losses, label="Validation Loss")
        plt.xlabel("Epoch")
        plt.ylabel("MSE Loss")
        plt.title("Training and Validation Loss over Epochs")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()


    return model, train_losses, val_losses


In [9]:
# (A) Load and preprocess data
noisy_data = np.load("noisy_images_small_1k.npy").astype(np.float32)
clean_data = np.load("clean_images_small_1k.npy").astype(np.float32)


noisy_profiles = noisy_data.mean(axis=2)
clean_profiles = clean_data.mean(axis=2)

# Normalize noisy profiles 
noisy_profiles, mean, std = normalize_data(noisy_profiles)
clean_profiles = (clean_profiles - mean) / std

# In Torch-Tensoren umwandeln
inputs = torch.tensor(noisy_profiles)
targets = torch.tensor(clean_profiles)
dataset = TensorDataset(inputs, targets)

# 80/20 Split für Training und Validierung
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Train the model (angepasst mit Anti-Overfitting)
model, train_losses, val_losses = train_model(train_loader, val_loader)

# Beispiel: Trainings- und Validierungsprofil (Noisy, Clean, Predicted)
train_sample = train_dataset[0]
val_sample = val_dataset[0]
plot_noisy_clean_predicted(model, train_sample[0], train_sample[1], mean, std, title="Training Sample")
plot_noisy_clean_predicted(model, val_sample[0], val_sample[1], mean, std, title="Validation Sample")

plot_loss_curves(train_losses, val_losses)

display_noisy_clean_images(noisy_data[0], clean_data[0], titles=["Noisy Image", "Clean Image"])

# Finaler Validierungs-MSE
model.eval()
val_loss = 0
criterion = nn.MSELoss()
with torch.no_grad():
    for x_batch, y_batch in val_loader:
        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        val_loss += loss.item()
val_loss /= len(val_loader)
print(f"\nFinal validation MSE: {val_loss:.4f}")


NameError: name 'normalize_data' is not defined